In [ ]:
import pandas as pd
import numpy as np

#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Читаем CSV в Pandas Dataframe
train_data = pd.read_csv("../input/titanic/train.csv")
test_data = pd.read_csv("../input/titanic/test.csv")

In [ ]:
# Объединим тестовую и тренировочную выборку для наилучшего представления о значениях признаков
all_data = train_data.append(test_data, ignore_index=True)
# Посчитаем индексы для последующего разделения объединенной выборки на тренировочную и тестовую
train_idx = len(train_data)
test_idx = len(all_data) - len(test_data)

In [ ]:
# Посмотрим на полученный датафрейм
all_data.head()

In [ ]:
# Посмотрим на описание датафрейма
all_data.info()

Имеем 1309 пассажиров: из них 891 относится к тренировочной выборке, для них определено поле Survived. В полях Имя, Пол, состав семьи, билет нет пропусков. Возраст не определен у 250 человек, попробуем восстановить его. Номер каюты определен менее чем у 30% пассажиров, это поле не дает нам никаких объективных и полезных знаний. 

In [ ]:
# Посмотрим на выживаемость в зависимости от класса и пола
train_data.groupby(["Pclass", "Sex"])["Survived"].value_counts(normalize=True)

Видим, что выживаемость напрямую связана с полом и классмо пассажира. Выживаемость женщин во всех классах выше, чем мужчин.Также выживаеость тем выше, чем класс пассажира

Для восстановления данных о возрасте используем имена пассажиров. Имена имеют обращение Мистер, Миссис и т.д. С помощью такого обращения можно определить к какой возрастной группе относится человек. Это поможет нам заменить пропуски на медиану возраста по данной возрастной группе.

In [ ]:
# Выделяем обращение из имени
all_data['NamePrefix'] = all_data["Name"].apply(lambda name: name.split(',')[1].split('.')[0].strip())

In [ ]:
# Посмотрим на полученные обращения
all_data['NamePrefix'].unique()

На борту титаника были пассажиры разных стран. Попробуем привести все обращения к одному виду: например Mme французкий эквивалент обращения Mrs. Создадим словарь для преобразования эквивалентых обращений:

In [ ]:
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}

all_data["NamePrefix"] = all_data["NamePrefix"].map(normalized_titles)
all_data["NamePrefix"].value_counts()

Теперь мы имеем отдельный признак для каждого пассажира: его статус в обществе через обращение к нему. Теперь посчитаем медианный возраст для каждой группы пассажиров в зависимости от их класса. 

In [ ]:
grouped = all_data.groupby(['Sex','Pclass', 'NamePrefix'])  
grouped["Age"].median()

In [ ]:
all_data["Age"] = grouped["Age"].apply(lambda x: x.fillna(x.median()))

In [ ]:
#Посмотрим на изменения в колонке Возраст
all_data.info()

Теперь мы не имеем пропуск в поле возраст. Это должно помочь при анализе, так как возраст пассаижра является одним из главных показателей выживаемости.

In [ ]:
# Заполним пропуск в графе Fare на медианное значение
all_data["Fare"] = all_data["Fare"].fillna(all_data["Fare"].median())

# Посмотрим на изменения
all_data.info()

In [ ]:
# Синтезируем новый признак: размер семьи. Это можно сделать на основании числа родителей и числа братьев/сестер на борту
all_data['FamilySize'] = all_data["Parch"] + all_data["SibSp"] + 1

Большим семьям было тяжело получить место в спастельной лодке, в отличие от одиноких пассажиров или маленьких семей. Попробуем синтезировать новый признак и избавить от двух имеющихся(Parch и SibSp).

In [ ]:
# Посмотрим на устройство получившего датафрейма
all_data.head()

In [ ]:
# Нужно закодировать пол пассаижров. Мужчинам будет соответсовать 0, женщин 1.
all_data["Sex"] = all_data["Sex"].map({"male": 0, "female":1})

In [ ]:
#Отбросим признаки, которые не будут участвовать в классификации: обращение к имени(мы его использовали для определния возраста),
# имя, билет, порт посадки, 
# братья/сестры и родители(их мы синтезировали в отдельный признак FamilySize), номер каюты(мало у кого определен)
all_data.drop(['NamePrefix', 'Cabin', 'Embarked','SibSp','Parch', 'Name', 'Ticket'], axis=1, inplace=True)
all_data.head()

Датасет готов для обучения и тестирования, можно начинать разделять его на тренировочную и тестовую части

In [ ]:
# Рзаделям исправленный датасет
train = all_data[ :train_idx]
test = all_data[test_idx: ]

# Создаем numpy array с тренировочной выборкой и целевую переменную
X = train.drop('Survived', axis=1).values 
y = train["Survived"].values
y = y.astype("int")

# Создаем numpy array для предсказания
X_test = test.drop('Survived', axis=1).values
X

In [ ]:
# Попробуем перебрать некоторые парметры случайного леса: максимальную глубинку дерева, 
# количесто деревьев в лесу и гиперпарметры разделения внутренних вершин дерева
forrest_params = dict(     
    max_depth = [n for n in range(9, 15)],     
    min_samples_split = [n for n in range(4, 11)], 
    min_samples_leaf = [n for n in range(2, 5)],     
    n_estimators = [n for n in range(10, 20, 1)],
)

In [ ]:
# создаем экемпляр Случайного леса
forrest = RandomForestClassifier()

In [ ]:
# Создаем класс для подрбора параметров случайного леса и обучаем модель
forest_cv = GridSearchCV(estimator=forrest, param_grid=forrest_params, cv=5) 
forest_cv.fit(X, y)

In [ ]:
print("Наилучший рузльтат: {}".format(forest_cv.best_score_))
print("Оптимальные параметры: {}".format(forest_cv.best_estimator_))

In [ ]:
# Используем обученную модель для предсказаний
forrest_pred = forest_cv.predict(X_test)

In [ ]:
# Формируем датафрейм с предксакзаниями
passengerId = test_data["PassengerId"]
result = pd.DataFrame({'PassengerId': passengerId, 'Survived': forrest_pred})

# сохранем результат в файл
result.to_csv('titanic.csv', index=False)